# Segmenting and clustering neighborhoods in Toronto

In [1]:
#installing folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



In [2]:
#importing the rest of the libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests

from bs4 import BeautifulSoup
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

Solving environment: done

# All requested packages already installed.



# Scraping the wikipedia page using the BeautifulSoup Libary


We use the request.get() fuction from the request library to retrieve the text from the wikipedia page. The retrieved html page is stored in the text attribute of the page object. We then use the BeautifulSoup Library to parse the page.

In [3]:
html = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(html)
text = page.text
soup = BeautifulSoup(text, 'html.parser')

   
Now we use the find() function to search for the table.
We can then extract the rows from the table and headers are extracted to retrieve the column names
This table along with the column names are then loaded in to a pandas dataframe

In [4]:
table = soup.find('table')
rows = table.findAll('tr')
columns = rows[0].findAll('th')
no_of_cols = len(columns)
no_of_rows = len(rows)
#since the row containg the column names is used we subtract a row from the total no of rows
no_of_rows = no_of_rows - 1

#now lets write the column headers into a list
cols= list()

for i in range(no_of_cols):
    if i == no_of_cols - 1:
        cols.append(columns[i].string[:-1].lower())
    else:
        cols.append(columns[i].string.lower())

df = pd.DataFrame(columns = cols, index = range(no_of_rows))

for i in range(no_of_rows):
    new_rows = rows[i+1].findAll('td')
    for j in range(no_of_cols):
        df.iloc[i,j] = new_rows[j].string
        if df.iloc[i,j] == None:
            df.iloc[i,j] = new_rows[j].a.string 
            
#remove the line breaks
for i in df.index:
    if df.loc[i,'neighbourhood'][-1:] == '\n':
        df.loc[i,'neighbourhood'] = df.loc[i,'neighbourhood'][:-1]
print(df.shape)
df.head(10)

(289, 3)


,postcode,borough,neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


   
# Preprocessing the dataframe 

Since some of the rows in the dataframe have no boroughs assigned to them they provide no required information so we should drop these rows

In [5]:
#lets see how many rows have boroughs that are not assigned
count = 0
for i in df.index:
    if df.loc[i,'borough'] == 'Not assigned':
        count = count + 1
count


77

There are 77 rows thats are not assigned a borough. Now we drop these rows and assign the borough to neighbourhoods that have boroughs assigned but have the neighbourhoods unassigned

In [6]:
for i in df.index:
    if df.loc[i,'borough'] == 'Not assigned':
        df = df.drop(i,axis=0)
    elif df.loc[i,'neighbourhood'] == 'Not assigned':
        df.loc[i,'neighbourhood'] = df.loc[i,'borough']
df.head()

,postcode,borough,neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [7]:
postcodes = list(df['postcode'].unique())
index=range(len(postcodes))
new_df = pd.DataFrame(columns=cols,index=range(len(postcodes)))

for i in new_df.index:
    code = postcodes[i]
    tmp_df = df[df['postcode']==code]
    hood_string = ''
    for j in range(tmp_df.shape[0]):
        hood_string = hood_string + tmp_df.loc[tmp_df.index[j],'neighbourhood'] + ','
    hood_string = hood_string[:-1]
    new_df.loc[i,'postcode'] = code
    new_df.loc[i,'borough'] = tmp_df.loc[tmp_df.index[0],'borough']
    new_df.loc[i,'neighbourhood'] = hood_string
    
new_df.reset_index(inplace=True)
new_df.drop('index',axis=1,inplace=True)
new_df.rename(columns = {'postcode' : 'postal code'},inplace = True)
new_df.head(10)

,postal code,borough,neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


Now lets see use the shape function to find the size of the above dataframe.

In [8]:
print('Number of Neighbourhoods are {} and number of boroughs are {} '.format(new_df.shape[0],len(new_df['borough'].unique())))

Number of Neighbourhoods are 103 and number of boroughs are 11 



___
Now to get the coordinates for the different postal codes, we use the data from the csv file from http://cocl.us/Geospatial_data.

In [9]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
## lets add the latitude and longitude colums to our dataframe
new_df['latitude'] = ''
new_df['longitude'] = ''

#Now lets fill these columns with the coordinates we obtained in geo_df
for i in new_df.index:
    new_df.loc[i,'latitude'] = geo_df[geo_df['Postal Code'] == new_df.loc[i,'postal code']].iloc[0,1]
    new_df.loc[i,'longitude'] = geo_df[geo_df['Postal Code'] == new_df.loc[i,'postal code']].iloc[0,2]

neighborhoods = new_df
neighborhoods.head()

,postal code,borough,neighbourhood,latitude,longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6543,-79.3606
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.7185,-79.4648
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895


Now we get the location of Toronto using the Nominatim() function

In [16]:
toronto_map = folium.Map(location=[neighborhoods.latitude.mean(),neighborhoods.longitude.mean()],zoom_start=10)

for lat,lng,borough,code in zip(neighborhoods['latitude'],neighborhoods['longitude'],neighborhoods['borough'],neighborhoods['postal code']):
    label = '{}, {}'.format(code,borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7,parse_html=False).add_to(toronto_map)  
    
toronto_map


Now we can chose a borough we would like to explore. Lets explore the Downtown Toronto borough

In [23]:
Downtown_Toronto = neighborhoods[neighborhoods['borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto

,postal code,borough,neighbourhood,latitude,longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6543,-79.3606
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3789
2,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
3,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733
4,M5G,Downtown Toronto,Central Bay Street,43.658,-79.3874
5,M6G,Downtown Toronto,Christie,43.6695,-79.4226
6,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.6506,-79.3846
7,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.6408,-79.3818
8,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.6472,-79.3816
9,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.6482,-79.3798


Now lets visualize the neighborhoods in Downtown toronto

In [28]:
Downtown_Toronto_map = folium.Map(location=[Downtown_Toronto.latitude.mean(),Downtown_Toronto.longitude.mean()],zoom_start=13)

for lat,lng,borough,code in zip(Downtown_Toronto['latitude'],Downtown_Toronto['longitude'],Downtown_Toronto['borough'],Downtown_Toronto['postal code']):
    label = '{}, {}'.format(code,borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7,parse_html=False).add_to(Downtown_Toronto_map)  
    
Downtown_Toronto_map

Lets get the geographical coordinates of Downtown Toronto

In [21]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [17]:
CLIENT_ID = 'KKXTSUNT2EBBQCTDDYKZHFFJI2S1QBNMADIDFA3IBMQV0MY2' 
CLIENT_SECRET = 'MZFRDDVMZQEWNSPT4KG04ULHHTH5WVTQTCLC3CHMGZSYC124' 
VERSION = '20180105' 

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: KKXTSUNT2EBBQCTDDYKZHFFJI2S1QBNMADIDFA3IBMQV0MY2
CLIENT_SECRET:MZFRDDVMZQEWNSPT4KG04ULHHTH5WVTQTCLC3CHMGZSYC124


Lets define a function to get the venues for each of the neighborhood

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Downtow_Toronto_venues = getNearbyVenues(names=Downtown_Toronto['neighbourhood'],
                                   latitudes=Downtown_Toronto['latitude'],
                                   longitudes=Downtown_Toronto['longitude']
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city
Church and Wellesley


In [34]:
print(Downtown_Toronto_venues.shape)
Downtown_Toronto_venues.head()

(1287, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [38]:
Downtown_Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,54,54,54,54,54,54
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",48,48,48,48,48,48
Central Bay Street,85,85,85,85,85,85
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,88,88,88,88,88,88
"Commerce Court,Victoria Hotel",100,100,100,100,100,100


### Analyze each Neighborhood

In [41]:
# one hot encoding
onehot = pd.get_dummies(Downtown_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = Downtown_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]
print(onehot.shape)
onehot.head()

(1287, 207)


,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [43]:
downtown_grouped = onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.010000,0.010000,0.000000,0.03,0.0000,0.000000,0.0000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.020000,0.010000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.010000,0.060000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.000000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.000000,0.000000,0.01,0.00,0.000000,0.01,0.00,0.010000,0.000000,0.030000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.00,0.010000,0.010000,0.00,0.000000,0.000000,0.020000,0.010000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.010

___
Lets print the top 5 venues of each neighborhood

In [44]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3           Steakhouse  0.04
4      Thai Restaurant  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.06
2      Restaurant  0.06
3  Farmers Market  0.04
4     Cheese Shop  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3             Plane  0.07
4     Boat or Ferry  0.07


----Cabbagetown,St. James Town----
                venue  freq
0          Restaurant  0.08
1         Coffee Shop  0.08
2  Italian Restaurant  0.04
3                 Pub  0.04
4         Pizza Place  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.07
2  Italian Restaurant  0.05
3      Ice Cream

Let's put that into a pandas dataframe   
First, let's write a function to sort the venues in descending order.


In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Lets find the top 5 venues of each neighborhood**

In [48]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Pub,Bakery
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boutique
3,"Cabbagetown,St. James Town",Restaurant,Coffee Shop,Pharmacy,Indian Restaurant,Pizza Place
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint
5,"Chinatown,Grange Park,Kensington Market",Café,Bar,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Vietnamese Restaurant
6,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant
7,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint
8,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant
9,"Design Exchange,Toronto Dominion Centre",Coffee Shop,Hotel,Café,Restaurant,American Restaurant


## Clustering 

Lets cluster the neighborhoods in Downtown Toronto into 6 clusters using K-Means

In [91]:
kclusters = 6
downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)
kmeans.labels_

array([4, 2, 5, 2, 0, 4, 3, 4, 0, 0, 0, 4, 0, 2, 1, 4, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood

In [92]:
downtown_merged = Downtown_Toronto
downtown_merged.drop('borough',axis=1)
downtown_merged['Cluster Labels'] = kmeans.labels_
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighbourhood')
downtown_merged.head() 

,postal code,borough,neighbourhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6543,-79.3606,4,Coffee Shop,Park,Café,Pub,Bakery
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3789,2,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant
2,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754,5,Coffee Shop,Restaurant,Hotel,Café,Cocktail Bar
3,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733,2,Coffee Shop,Restaurant,Cocktail Bar,Pub,Bakery
4,M5G,Downtown Toronto,Central Bay Street,43.658,-79.3874,0,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint


#### Cluster visualization

In [94]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for code,lat, lon, poi, cluster in zip(downtown_merged['postal code'],downtown_merged['latitude'], downtown_merged['longitude'], downtown_merged['neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Now lets examine the clusters

In [95]:
#Cluster 1
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint
8,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,American Restaurant
9,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant
10,Downtown Toronto,0,Café,Bar,Coffee Shop,Bookstore,Japanese Restaurant
12,Downtown Toronto,0,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boutique


In [96]:
#Cluster 2
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Pub,Hotel


In [97]:
#Cluster 3
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Downtown Toronto,2,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant
3,Downtown Toronto,2,Coffee Shop,Restaurant,Cocktail Bar,Pub,Bakery
13,Downtown Toronto,2,Park,Playground,Trail,Women's Store,Deli / Bodega
16,Downtown Toronto,2,Café,Coffee Shop,Hotel,Restaurant,American Restaurant
17,Downtown Toronto,2,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint


In [98]:
#Cluster 4
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,Downtown Toronto,3,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant


In [99]:
#Cluster 5
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Downtown Toronto,4,Coffee Shop,Park,Café,Pub,Bakery
5,Downtown Toronto,4,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant
7,Downtown Toronto,4,Coffee Shop,Aquarium,Hotel,Café,Pizza Place
11,Downtown Toronto,4,Café,Bar,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Vietnamese Restaurant
15,Downtown Toronto,4,Restaurant,Coffee Shop,Pharmacy,Indian Restaurant,Pizza Place


In [101]:
#Cluster 6
downtown_merged.loc[downtown_merged['Cluster Labels'] == 5, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Downtown Toronto,5,Coffee Shop,Restaurant,Hotel,Café,Cocktail Bar
